In [61]:
import pandas as pd
import numpy as np
import os

In [107]:
PATH_HTS_PERSONS_12_16_SA1_V1 = "inputs/melbourne/hts_12_16/P_VISTA12_16_SA1_V1.csv"
PATH_HTS_HOUSEHOLDS_12_16_SA1_V1 = "inputs/melbourne/hts_12_16/H_VISTA12_16_SA1_V1.csv"
GEO_FILE = "inputs/melbourne/geography/MB_2016_VIC.csv"
SA1_2011_2016 = 'inputs/melbourne/geography/sa1_2011_2016.xlsx'
PERSONS_ATTRIBUTES = [
        "PERSID",
        "AGE",
        "SEX",
        "FULLTIMEWORK",
        "PARTTIMEWORK",
        "CASUALWORK",
        "ANYWORK",
        "HHID",
        "CW_ADPERSWGT_SA3",
        "HomeSA1"
    ]
HOUSEHOLDS_ATTRIBUTES = [
    "HHID", 
    "HHSIZE", 
    "CARS", 
    "TOTALVEHS", 
    "CW_ADHHWGT_SA3",
    "HomeSA1"
    ]

In [108]:
def mapping_SA1_to_rest(df_target, geo_df):
    zip_from_geo = zip(
        geo_df['SA1_MAINCODE_2016'],
        geo_df['SA1_7DIGITCODE_2016'],
        geo_df['SA2_MAINCODE_2016'],
        geo_df['SA3_CODE_2016'],
        geo_df['SA4_CODE_2016']
    )

    dict_map = {}
    for sa1_full, sa1_7digit, sa2, sa3, sa4 in zip_from_geo:
        dict_map[sa1_full] = {
            'SA1': sa1_7digit,
            'SA2': sa2,
            'SA3': sa3,
            'SA4': sa4
        }

    SA1_arr, SA2_arr, SA3_arr, SA4_arr = [], [], [], []
    for sa1_full in df_target['HomeSA1']:
        if sa1_full in dict_map:
            val = dict_map[sa1_full]
            SA1_arr.append(val['SA1'])
            SA2_arr.append(val['SA2'])
            SA3_arr.append(val['SA3'])
            SA4_arr.append(val['SA4'])
        else:
            SA1_arr.append(None)
            SA2_arr.append(None)
            SA3_arr.append(None)
            SA4_arr.append(None)

    df_target['SA1'] = SA1_arr
    df_target['SA2'] = SA2_arr
    df_target['SA3'] = SA3_arr
    df_target['SA4'] = SA4_arr
    return df_target

In [109]:
geo_cross_walk = pd.read_csv(GEO_FILE)

In [110]:
sa1_2011_2016 = pd.read_excel(SA1_2011_2016, dtype=int)
sa1_2011_2016_map = dict(zip(sa1_2011_2016['SA1_MAINCODE_2011'], sa1_2011_2016['SA1_MAINCODE_2016']))

In [111]:
households_seed = pd.read_csv(PATH_HTS_HOUSEHOLDS_12_16_SA1_V1)

households_seed = households_seed[HOUSEHOLDS_ATTRIBUTES]
households_seed["CARS"] = np.where(households_seed["CARS"] == 0, "No", "Yes")
households_seed = households_seed.reset_index(drop=True)
households_seed['hhnum'] = households_seed.index + 1
households_seed['HomeSA1'] = households_seed['HomeSA1'].replace(sa1_2011_2016_map)
households_seed = mapping_SA1_to_rest(households_seed,geo_cross_walk)

households_seed

/var/folders/fg/wfh8sp4x7lbb49hgnww7yrlm0000gn/T/ipykernel_65963/1266155506.py:1: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  households_seed = pd.read_csv(PATH_HTS_HOUSEHOLDS_12_16_SA1_V1)


,HHID,HHSIZE,CARS,TOTALVEHS,CW_ADHHWGT_SA3,HomeSA1,hhnum,SA1,SA2,SA3,SA4
0,Y12H0000101,4,Yes,2,91.34,20904122028,1,2122028,209041220,20904,209
1,Y12H0000102,4,Yes,3,91.26,20904122028,2,2122028,209041220,20904,209
2,Y12H0000103,4,Yes,2,91.03,20904122028,3,2122028,209041220,20904,209
3,Y12H0000104,3,Yes,3,91.81,20904122028,4,2122028,209041220,20904,209
4,Y12H0000107,3,Yes,1,107.80,20904122028,5,2122028,209041220,20904,209
...,...,...,...,...,...,...,...,...,...,...,...
18147,Y16H2080525,6,No,2,230.15,21304135805,18148,2135805,213041358,21304,213
18148,Y16H2080531,1,No,1,228.19,21304135805,18149,2135805,213041358,21304,213
18149,Y16H2080532,2,Yes,1,230.15,21304135805,18150,2135805,213041358,21304,213
18150,Y16H2080533,1,No,0,231.13,21304135805,18151,2135805,213041358,21304,213


In [112]:
persons_seed = pd.read_csv(PATH_HTS_PERSONS_12_16_SA1_V1)
persons_seed = persons_seed[PERSONS_ATTRIBUTES]

persons_seed.loc[persons_seed["CASUALWORK"] == "Yes", "ANYWORK"] = "CASUALWORK"
persons_seed.loc[persons_seed["PARTTIMEWORK"] == "Yes", "ANYWORK"] = "PARTTIMEWORK"
persons_seed.loc[persons_seed["FULLTIMEWORK"] == "Yes", "ANYWORK"] = "FULLTIMEWORK"
persons_seed.loc[persons_seed["ANYWORK"] == "No", "ANYWORK"] = "UNEMPLOYED"
persons_seed = persons_seed.drop(
    columns=["FULLTIMEWORK", "PARTTIMEWORK", "CASUALWORK"]
)

In [113]:
assert len(persons_seed) == len(persons_seed[persons_seed.HHID.isin(households_seed.HHID)]),print("Please remove person thatare not in household seed")
households_num_dict = dict(zip(households_seed.HHID, households_seed.hhnum))
persons_seed['hhnum'] = persons_seed['HHID'].map(households_num_dict)

In [114]:
persons_seed['HomeSA1'] = persons_seed['HomeSA1'].replace(sa1_2011_2016_map)
persons_seed = mapping_SA1_to_rest(persons_seed,geo_cross_walk)

In [115]:
persons_seed.ANYWORK.value_counts()

UNEMPLOYED      22744
FULLTIMEWORK    15690
PARTTIMEWORK     5169
CASUALWORK       2959
Name: ANYWORK, dtype: int64

In [116]:
households_seed.to_csv('outputs/seed_households.csv')
persons_seed.to_csv('outputs/seed_persons.csv')